# A tutorial on model validation using deep generation of stress data

<img src="https://raw.githubusercontent.com/airi-industrial-ai/dsc23-tutorial/8e761b9a2851a923970afb548275aebffc480a55/images/airi_logo.png" width=200 align='left'/>

_Vitaliy Pozdnyakov, Junior Research Scientist | AIRI_

Open the notebook by the following QR code:

<img src="https://raw.githubusercontent.com/airi-industrial-ai/dsc23-tutorial/8e761b9a2851a923970afb548275aebffc480a55/images/qr_code.png" width=200 align='left'/>

## Our plan for today

1. Theory: Dataset shifts
2. Practice: Download and prepare a dataset for time series forecasting
3. Practice: Train and test a LSTM-based forecasting model
4. Theory: Worst-case risk
5. Practice: Evaluate the stability of the LSTM-based model using the worst-case risk
6. Theory: Time series generation
7. Practice: Generate fake time series similar to the original dataset
8. Practice: Evaluate the stablity of the LSTM-based model using the worst-case risk on fake data

## Dataset shifts in the real setting

Let's consider some predictive model, say a recommender system. In the ideal world we expect the following process: we train the model on historical data, then we run it on actual data and see the great results in terms of some metrics (MSE, Cross-entropy, etc).

<img src="https://raw.githubusercontent.com/airi-industrial-ai/dsc23-tutorial/8e761b9a2851a923970afb548275aebffc480a55/images/ideal_setting.png" width=800 align='left'/>

**The main assumption**: train and test datasets are sampled from the same distributions or at least similar distributions. 

In reality, we often face some **challenges** in the real world setting:
* data usually changes over time
* shocks affect dramatically

<img src="https://raw.githubusercontent.com/airi-industrial-ai/dsc23-tutorial/8e761b9a2851a923970afb548275aebffc480a55/images/challenges.png" width=450 align="left"/>

### Obviously, models fail when data is changed, do not they?

The problem of changed data is known as **dataset (distribution) shift**. We consider two types of distribution shifts: concept shifts and covariate shifts.

**Concept shift** is the shift in the conditional distribution of target by given features while the distribution of features remains the same:

$$P_{tr}(X) = P_{tst}(X)$$
$$P_{tr}(y|X) \neq P_{tst}(y|X)$$

Example: 𝑋 is a message, 𝑦 is a spam marker. We test the model on a new type of spam.

In the case of the concept shift we actually need a new model. The following techinques are usually used: retraining, fine-tuning, online learning.

**Covariate shift** is the shift in the distribution of features while the conditional distribution of target by given features remains the same:

$$P_{tr}(X) \neq P_{tst}(X)$$
$$P_{tr}(y|X) = P_{tst}(y|X)$$

Example: 𝑋 is medical data, 𝑦 is a status of a disease. We test the model at another hospital.

In the case of the covariate shift, we need to be sure that our model is robust to these shifts.

<img src="https://raw.githubusercontent.com/airi-industrial-ai/dsc23-tutorial/88453d7be9ef5bbb791f67df3697701639f5784c/images/shifts.png" width=600 align='left'/>

This phenomenon of dataset shift in time is widespread across various domains:

<img src="https://raw.githubusercontent.com/airi-industrial-ai/dsc23-tutorial/8e761b9a2851a923970afb548275aebffc480a55/images/data_description.png" width=800 align="left"/>

_Credits: Yao, Huaxiu, et al. "Wild-time: A benchmark of in-the-wild distribution shift over time." Advances in Neural Information Processing Systems 35 (2022): 10309-10324._

In this tutorial, we will learn how to evaluate the model robustness to covariate dataset shift and then will look at how the generative models can be used in the evaluation.

## Dataset for time series forecasting

In [ ]:
!pip install git+https://github.com/airi-industrial-ai/dsc23-tutorial
!pip install git+https://github.com/airi-industrial-ai/genrisk

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import trange, tqdm
import numpy as np
import requests
from scipy.stats import ks_2samp, norm
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.stattools import pacf, acf

In [ ]:
from genrisk.shift import ConditionalShift
from genrisk.generation import TCNGAN, LSTMGAN

In [ ]:
from dsctutorial.data import load_gas_supply
from dsctutorial.utils import positional_encoding, load_from_checkpoint
from dsctutorial.plot import hist2samp, pacf2samp
from dsctutorial.forecast import LSTMForecaster

In [ ]:
import warnings
warnings.filterwarnings("ignore")

The dataset is represented by weekly U.S. product supplied of finished motor gasoline (in thousand barrels per day). The dataset is available on the page https://www.eia.gov/dnav/pet/hist/LeafHandler.ashx?n=PET&s=wgfupus2&f=W.

In [ ]:
target = load_gas_supply()
target

In [ ]:
split = target.index[int(len(target) * 0.7)]

plt.figure(figsize=(10, 3))
plt.vlines(x=split, ymin=target.min(), ymax=target.max(), color='tab:orange', label='train-test split')
plt.plot(target)
plt.legend()
plt.show()

Additionally, we consider some **covariates** which known in advance and help our model to predict the future. Here we will use the simple positional ecnoding of the week of year:
$$
\text{pos. encoding}(i) = \left(\sin \frac{i2\pi}{52} , \cos \frac{i2\pi}{52} \right)
$$
where $i$ is the number of a week which is in the range from 0 to 52.

In [ ]:
cov = positional_encoding(target.index, ['weekofyear'])
plt.figure(figsize=(10, 3))
plt.plot(cov)
plt.show()

Let us split our target and covariates into train and test samples.

In [ ]:
train_target, test_target = target[:split], target[split:]
scaler = StandardScaler()
train_target[:] = scaler.fit_transform(train_target)
test_target[:] = scaler.transform(test_target)
train_cov, test_cov = cov[:split], cov[split:]
len(train_target), len(test_target)

Let us consider the train and test target data distribution. It is the common practice in time series data to compare the distribution of the difference between subsequent datapoints:
$$\text{diff}_i = x_i - x_{i-1}$$

In [ ]:
hist2samp(train_target, test_target, 'train', 'test', nbins=30)

We can see that some slight dataset shift is introduced in the test sample.

## LSTM forecasting model

<img src="https://raw.githubusercontent.com/airi-industrial-ai/dsc23-tutorial/8e761b9a2851a923970afb548275aebffc480a55/images/lstm.png" width=600 align="left"/>

Let us train a simple LSTM model to predict the target time series.

In [ ]:
"""
lstm_model = LSTMForecaster(hidden_dim=64, window_size=100, lr=0.01, num_epochs=30, batch_size=32)
lstm_model.fit(train_target, train_cov)
""";

Here we don't waste time for training a neural network, insted we download a pretrained model.

In [ ]:
url = 'https://github.com/airi-industrial-ai/dsc23-tutorial/raw/main/weights/lstm-epoch=14-step=450.ckpt'
r = requests.get(url)
open('lstm-epoch=14-step=450.ckpt', 'wb').write(r.content);

In [ ]:
lstm_model = LSTMForecaster(hidden_dim=64, window_size=100, lr=0.01, num_epochs=15, batch_size=32)
lstm_model.load_from_checkpoint(
    'lstm-epoch=14-step=450.ckpt',
    train_target,
    train_cov,
)

Let us look how the model predict the future on test sample.

In [ ]:
pred = lstm_model.predict(len(test_target), train_target, train_cov, test_cov)

plt.figure(figsize=(10, 3))
plt.plot(test_target, label='test target')
plt.plot(pred, label='forecast')
plt.legend()
plt.show()

## Backtesting of forecasting models

Making prediction for 9 years ahead forecasting is nonsense. Instead, the common practice in testing forecasting models is **backtesting**. Backtesting is the procedure which evaluates the model in the historical order using the sliding window.

<img src="https://raw.githubusercontent.com/airi-industrial-ai/dsc23-tutorial/8e761b9a2851a923970afb548275aebffc480a55/images/sliding_window.png" width=800 align='left'/>

We split each window into past range and future range with respect to input size and forecasting horizon. We will use the past target, past covariates and future covariates to forecast the future target.

In [ ]:
def backtest(model, input_size, horizon, target, cov):
    error = []
    for i in trange(len(target)-input_size-horizon+1):
        past_range = range(i, i+input_size)
        future_range = range(i+input_size, i+input_size+horizon)
        past_target = target.iloc[past_range]
        past_cov = cov.iloc[past_range]
        future_cov = cov.iloc[future_range]
        future_target = target.iloc[future_range]
        future_pred = model.predict(horizon, past_target, past_cov, future_cov)
        error.append(((future_target.values[-1] - future_pred.values[-1])**2).mean())
    return pd.Series(error, index=target.index[input_size-1:-horizon])

We will use the followin forecasting problem:
* input size is 10 weeks
* forecasting horizon is 100 weeks

In [ ]:
train_lstm_error = backtest(lstm_model, input_size=10, horizon=100, target=train_target, cov=train_cov)
test_lstm_error = backtest(lstm_model, input_size=10, horizon=100, target=test_target, cov=test_cov)
train_lstm_error.mean(), test_lstm_error.mean()

## How to estimate the possible risks?

**Idea: some variables can change, other are fixed.**

* __Immutable variables__ keep their distribution unchanged while dataset shifts occur (e.g. demographic characteristics)
* __Mutable variables__ can have changes (e.g. treatment plan)

Notation:
* $X = W \cup Z$
* $W$ — mutable variables 
* $Z$ — immutable variables 

Original data: $P_{tr}(X) = P(W|Z)P(Z)$

Shifted data: $P_{tst}(X)=Q(W|Z)P(Z)$

**Worst-case risk** is the method to esimate potential decrease of the quality due to dataset shift. The steps of estimation:
1. Find the small fraction of the data which the model predicts the worst
2. Make sure the distribution of immutable variables is (almost) the same
3. Evaluate the model on this subsample

**The smaller the fraction, the higher the risk and vice versa.** The size of the fraction is denotes as $(1-\alpha)$.

To do these steps we use two models:
1. $\mu$: __Expectation model__ takes both mutable and immutable variables and estimate the expected loss of the target model
2. $\eta$: __Quantile model__ takes immutable variables and estimate the $\alpha$-quantile of the loss of the target model
3. $x$ is in the worst subsample if $\mu(w, z) > \eta(z)$

<img src="https://raw.githubusercontent.com/airi-industrial-ai/dsc23-tutorial/8e761b9a2851a923970afb548275aebffc480a55/images/wcr.png" width=800 align='left'/>

_Credits: Subbaswamy, Adarsh, Roy Adams, and Suchi Saria. "Evaluating model
    robustness and stability to dataset shift." International Conference on Artificial
    Intelligence and Statistics. PMLR, 2021._

Let us create a dataset which we use to define mutable and immutable variables

In [ ]:
def input_seq_dataset(input_size, horizon, target, cov, lags):
    input_seq = []
    for i in trange(len(target)-input_size-horizon+1):
        past_range = range(i, i+input_size)
        future_range = range(i+input_size, i+input_size+horizon)
        past_target = target.iloc[past_range]
        past_cov = cov.iloc[past_range]
        past_target_lags = []
        for j in range(lags):
            past_target_lags.append(past_target.iloc[-1, 0] - past_target.iloc[-2-j, 0])
        input = np.concatenate([past_target_lags, past_cov.iloc[-1].values])
        input_seq.append(input)
    columns = [f'{target.columns[0]}_diff{i+1}' for i in range(lags)] + list(cov.columns)
    input_seq = pd.DataFrame(input_seq, columns=columns, index=target.index[input_size-1:-horizon])
    return input_seq

In [ ]:
test_input_seq = input_seq_dataset(
    input_size=10, horizon=100, target=test_target, cov=test_cov, lags=5)
test_input_seq

Let the variables with diffs be mutable and weekofyear cos and sin be immutable variables. 

In [ ]:
mutable_columns = test_input_seq.columns[:-2].tolist()
immutable_columns = test_input_seq.columns[-2:].tolist()
mutable_columns, immutable_columns

We estimate the worst-case risk on different levels of risk: from 0.1 to 0.9.

In [ ]:
alphaspace = np.linspace(0.1, 0.9, 9)
test_lstm_risk = []
for alpha in tqdm(alphaspace):
    shift_model = ConditionalShift(mutable_columns, immutable_columns, alpha=alpha)
    shift_model.fit(test_input_seq, test_lstm_error)
    test_lstm_risk.append((shift_model.risk, shift_model.lb_risk, shift_model.ub_risk))
test_lstm_risk = np.array(test_lstm_risk)

We plot risk values and 95% confidence interval.

In [ ]:
plt.plot(alphaspace, test_lstm_risk[:, 0], c='tab:green')
plt.fill_between(alphaspace, test_lstm_risk[:, 1], test_lstm_risk[:, 2], alpha=0.2, color='tab:green')
plt.hlines(test_lstm_error.mean(), alphaspace[0], alphaspace[-1], color='tab:blue', label='mean error')
plt.legend()
plt.title('Worst-case risk evaluation')
plt.xlabel('alpha, risk level')
plt.ylabel('MSE')
plt.show()

The model stores the mask for the worst subsample.

In [ ]:
shift_model.mask[:100]

Let us plot some example from the worst subsample (stress data).

In [ ]:
risk_date = test_input_seq[shift_model.mask].index[0]
risk_idx = np.where(test_target.index == risk_date)[0][0]

plt.figure(figsize=(10, 3))
plt.plot(test_target.iloc[risk_idx-10:risk_idx+100])
plt.show()

**Limitations** of the worst-case risk:
* High uncertainty in evaluation of risk with high $\alpha$
* Not all possible shifts are presented

The suggestion: let us generate new data to model covariate shifts!

## Time series generation using GAN

The difference between generative and discriminative models:

<img src='https://raw.githubusercontent.com/airi-industrial-ai/dsc23-tutorial/8e761b9a2851a923970afb548275aebffc480a55/images/gen_vs_disc.png' width=800 align='left'>

*Credits: https://medium.com/@jordi299/about-generative-and-discriminative-models-d8958b67ad32*

| Discriminative models | Generative models |
|:---|:---|
| Aims to predict the label by the given example| Aims to learn the full data distribution by the given subsample |
| Learns the probability $y \sim P(y|X)$ | Learns the probability $X,y \sim P(X, y)$ or $X \sim P(X)$ |
| _Examples of tasks:_ | _Examples of tasks:_ |
| 1. Is a cat on the photo? | 1. Create a photo of a cat that looks like the average of two given cat photos |
| 2. How old is a person on the photo? | 2. Create a photo of an avocado armchair |
| 3. What is the most likely number of tourists the next mounth? | 3. Write an essay on the topic of global warming |
| _Examples of models:_ | _Examples of models:_ |
| Decision Tree, Support Vector Machine (SVM), Convolutional Neural Network (CNN) | Autoregressive model (AR), Generative Adversarial Network (GAN), Variational Autoencoder (VAE), Generative Pretrained Transformer (GPT)|


Deep generative models are based on deep neural networks.

<img src='https://raw.githubusercontent.com/airi-industrial-ai/dsc23-tutorial/8e761b9a2851a923970afb548275aebffc480a55/images/space_generator.png' width=800 align='left'>

Generative Adversarial Network is a generative model that can be used to generate time series.

<img src='https://raw.githubusercontent.com/airi-industrial-ai/dsc23-tutorial/8e761b9a2851a923970afb548275aebffc480a55/images/gan_for_ts.png' width=800 align='left'>

A Generative Adversarial Network (GAN) is a type of model that consists of two neural networks: a generator and a discriminator.

* The objective of a generator (G) is to create realistic data
* The objective of a discriminator (D) is to detect fake data

We note the multivariate time series as $X_1, X_2, \dots, X_N$, where $X_t \in R^d$

The general objective of GAN is

$$\min_G \max_D V(D, G) = \mathbb E_{X} \log(D(X)) + \mathbb E_{Z} \log(1 - D(G(Z))$$

In this tutorial we use GAN where a generator and a discriminator are Temporal Convolutional Networks (TCN).

<img src='https://raw.githubusercontent.com/airi-industrial-ai/dsc23-tutorial/8e761b9a2851a923970afb548275aebffc480a55/images/tcn.png' width=700 align='left'>

_Credits: Oord, Aaron van den, et al. "Wavenet: A generative model for raw audio." arXiv preprint arXiv:1609.03499 (2016)._

The main advantages of TCN:
* Exponentially large receptive field — TCN can detect long and short patterns
* Can be applied to time series of arbitrary length
* Causal convolution prevents leakage from the future

The key parameter of TCN is the number of channels (also known as filters, kernels).

For generator:
* Input channels: latent space dimensionality
* Hidden channels: some large number to model temporal relationships
* Output channels: time series dimensionality

For discriminator:
* Input channels: time series dimensionality
* Hidden channels: some large number to model temporal relationships
* Output channels: 1 (the probability that the data is fake)

In [ ]:
train_data = pd.concat([train_target, train_cov], axis=1)
test_data = pd.concat([test_target, test_cov], axis=1)

Let us define our TCN GAN:
* `latent_dim` — the dimensionality of latent space which is Miltivariate Normal Distribution.
* `hidden_dim` — the number of channels/filters/kernels in the hidden layers
* `target_dim` — the dimenstionality of time series
* `num_layers` — the number of TCN layers
* `lr` — the learning rate for the optimization step

The values of parameters can be tuned using some search strategies, such as grid or random search.

Now we can start to train GAN. We will use test data for training GAN as we are looking for shifts near the test period.

In [ ]:
"""
gan = TCNGAN(
    target_columns=target.columns,
    conditional_columns=cov.columns,
    window_size=100,
    num_epochs=400,
    num_layers=1,
    hidden_dim=64,
    latent_dim=2,
    verbose=True,
    lr=0.01,
)
gan.fit(test_data)
""";

GAN is known as an unstable model in training, since the generator and discriminator fluctuate without a guarantee of convergence. Thus, it is strongly recommended to monitor auxiliary metrics to determine the total number of training steps, e.g. KS-distance or EMD.

Here we load the pretrained model from a `checkpoint` file.

In [ ]:
url = 'https://github.com/airi-industrial-ai/dsc23-tutorial/raw/main/weights/gan-epoch=399-step=26000.ckpt'
r = requests.get(url)
open('gan-epoch=399-step=26000.ckpt', 'wb').write(r.content);

In [ ]:
gan = TCNGAN(
    target_columns=target.columns,
    conditional_columns=cov.columns,
    window_size=100,
    num_epochs=400,
    num_layers=1,
    hidden_dim=64,
    latent_dim=2,
    verbose=True,
    lr=0.01,
)
load_from_checkpoint(gan, 'gan-epoch=399-step=26000.ckpt')

## Evaluation of generated time series

Visual evaluation by time series plots:

In [ ]:
plt.figure(figsize=(12, 3))
fakes = gan.sample(test_data, n_samples=5)
for fake in fakes:
    fake_target = fake[target.columns]
    fake_cov = fake[cov.columns]
    plt.plot(fake_target, alpha=0.5)
plt.plot(test_target, c='black')
plt.show()

Visual evaluation by histograms of difference:

In [ ]:
hist2samp(test_target, fake_target, 'test', 'fake', nbins=30)

Visual evaluation by auto-correlation functions of time series:

In [ ]:
pacf2samp(test_target, fake_target, 'test', 'fake', nlags=30)

Statistical testing:

In [ ]:
ks_2samp(
    (fake_target - fake_target.shift(1)).supply, 
    (test_target - test_target.shift(1)).supply
).pvalue

Comparison with baselines:

In [ ]:
def pacf_error(target0, target1, nlags=10):
    pacf0 = acf(target0, nlags=nlags)[1:]
    pacf1 = acf(target1, nlags=nlags)[1:]
    error = (pacf0 - pacf1)**2
    return error.mean()

In [ ]:
pacf_error(test_target, fake_target)

In [ ]:
noise = np.random.randn(len(test_target))
pacf_error(test_target, noise)

We can see that the fake time series are similar to real ones. For more information, see, for example, Wiese, Magnus, et al. "Quant GANs: deep generation of financial time series." Quantitative Finance 20.9 (2020): 1419-1440.

Conclusions so far:
* We split the dataset into train, test sets
* We trained GAN on the test set, as we are looking for shifts close to the test period
* We created fake time series and visually checked that they are similar to the real ones

## Worst-case risk on fake data

Generative models has generalization property: they can generate plausible examples that are invisible in real data. We can use it to find new covariate shifts in the original dsitribution.

<img src='https://raw.githubusercontent.com/airi-industrial-ai/dsc23-tutorial/8e761b9a2851a923970afb548275aebffc480a55/images/wrc_fake.png' width=700 align='left'>

Let us generate data using GAN and estimate worst-case risk on fake data. 

In [ ]:
fake_index = pd.date_range(
    start=test_target.index[0],
    periods=len(test_target)*5,
    freq=test_target.index.freq
)
fake_cov = positional_encoding(fake_index, ['weekofyear'])
fake_target = gan.sample(fake_cov, n_samples=1)[0][target.columns]

In [ ]:
fake_lstm_error = backtest(lstm_model, input_size=10, horizon=100, target=fake_target, cov=fake_cov)
fake_lstm_error.mean()

As we see, the quality on the fake set is close to the test set. It means that the model cannot detect much discrepancy between them.

We define mutable and immutable variables.

In [ ]:
fake_input_seq = input_seq_dataset(
    input_size=10, horizon=100,
    target=fake_target,
    cov=fake_cov,
    lags=5)
fake_input_seq

We estimate worst-case risk on fake data.

In [ ]:
alphaspace = np.linspace(0.1, 0.9, 9)
fake_lstm_risk = []
for alpha in tqdm(alphaspace):
    shift_model = ConditionalShift(mutable_columns, immutable_columns, alpha=alpha)
    shift_model.fit(fake_input_seq, fake_lstm_error)
    fake_lstm_risk.append((shift_model.risk, shift_model.lb_risk, shift_model.ub_risk))
fake_lstm_risk = np.array(fake_lstm_risk)

Let us compare estimated risk on test and fake data.

In [ ]:
plt.plot(alphaspace, test_lstm_risk[:, 0], c='tab:blue', label='test data')
plt.fill_between(alphaspace, test_lstm_risk[:, 1], test_lstm_risk[:, 2], alpha=0.2, color='tab:blue')

plt.plot(alphaspace, fake_lstm_risk[:, 0], c='tab:green', label='fake data')
plt.fill_between(alphaspace, fake_lstm_risk[:, 1], fake_lstm_risk[:, 2], alpha=0.2, color='tab:green')

plt.legend()
plt.title('Worst-case risk evaluation')
plt.xlabel('alpha, risk level')
plt.ylabel('MSE')

plt.show()

As we can see, we can get a narrower uncertainty interval for fake data. This means that we can more precisely assess the robustness of the model using fake data.

Finally, let us look at some generated stress examples.

In [ ]:
risk_date = fake_input_seq[shift_model.mask].index[0]
risk_idx = np.where(fake_target.index == risk_date)[0][0]

plt.figure(figsize=(10, 3))
plt.plot(test_target.iloc[risk_idx-10:risk_idx+100])
plt.show()

## Discussion

Generative models produce realistic data that can help estimate the model robustness to dataset shifts, however it brings some challenges.

1. It is hard to tune parameters and train deep generative models such GAN
2. It is hard to estimate how close the fake time series to real ones: no scientific standard so far
3. We assume that generative model generalizes original data distribution

## Conclusion

Today we've learned the following:

1. What is the dataset shifts
2. How to prepare a dataset for time series forecasting
3. How to train and test a LSTM-based forecasting model
4. What is the worst-case risk
5. How to evaluate the stability of the LSTM-based model using the worst-case risk
6. What is the time series generation
7. How to generate fake time series similar to the original dataset
8. How to evaluate the stablity of the LSTM-based model using the worst-case risk on fake data

### Feel free to write me on pozdnyakov@airi.net